# Fairify Artifact
Artifact of the paper "Fairify: Fairness Verification of Neural Networks" from ICSE 2023.

To access ChameleonCloud resources, you may need the account to log in to ChameleonCloud. You also need to have a project to allocate resources (e.g., node).

In [19]:
from chi import context

context.version = "1.0"

context.choose_site(default="CHI@TACC")
context.choose_project()

### Check available hardware

In [20]:
from chi import hardware

node_type = "compute_cascadelake_r"
available_nodes = hardware.get_nodes(node_type=node_type, filter_reserved=True)
if available_nodes:
    print(f"There currently are {len(available_nodes)} {node_type} nodes ready to use")
else:
    print(f"All {node_type} nodes are in use! You could use next_free_timeslot to see how long you need to wait, or use the calendar.")

There currently are 33 compute_cascadelake_r nodes ready to use


### Reserve node

In [21]:
from chi import lease
from datetime import timedelta
import os

my_lease = lease.Lease(f"{os.getenv('USER')}-power-management", duration=timedelta(hours=3))
my_lease.add_node_reservation(nodes=[available_nodes[0]]) # or you could use node_type=node_type
my_lease.add_fip_reservation(1) # include a floating ip
my_lease.submit(idempotent=True)

Waiting for lease to start... This can take up to 60 seconds


Lease radhofanazizi_gmail_com-power-management has reached status active


### Create a server on the node

In [22]:
from chi import server

my_server = server.Server(
    f"{os.getenv('USER')}-power-management",
    reservation_id=my_lease.node_reservations[0]["id"],
    image_name="CC-Ubuntu22.04", # or use image_name
)
my_server.submit(idempotent=True)

Waiting for server radhofanazizi_gmail_com-power-management's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,radhofanazizi_gmail_com-power-management
Id,d4dfe3f9-0b6f-4972-b1c0-66d047a9fe7c
Status,ACTIVE
Image Name,CC-Ubuntu22.04
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.3.93 (v4) Type: fixed MAC: bc:97:e1:78:f1:a0
Network Name,sharednet1
Created At,2025-05-18T07:38:32Z
Keypair,trovi-28d289c
Reservation Id,a381a41c-eec3-44c2-a657-8ad56aecff1e
Host Id,b281b13a05d4a4d342f673906de4005142c2819a049809e34ac97306


### Configure networking on the node

In [23]:
fip = my_lease.get_reserved_floating_ips()[0]
my_server.associate_floating_ip(fip)
my_server.check_connectivity(host=fip)

Checking connectivity to 129.114.109.220 port 22.


Connection successful


### Setup nvidia drivers and cuda

Download drivers

In [ ]:
my_server.execute("""
sudo apt-get --purge remove -y '*cublas*' 'cuda*' 'nvidia-*' && \
sudo apt-get autoremove -y && \
sudo apt-get autoclean && \
sudo rm -rf /usr/local/cuda* && \
sudo apt update && \
sudo apt install -y nvidia-driver-470 && \
wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-keyring_1.0-1_all.deb && \
sudo dpkg -i cuda-keyring_1.0-1_all.deb && \
sudo apt update && \
sudo apt install -y cuda-toolkit-11-2
""")

Restart nvidia services

In [ ]:
my_server.execute("sudo modprobe -r nouveau")
my_server.execute("sudo rmmod nouveau 2>/dev/null || true")
my_server.execute("sudo update-initramfs -u")
my_server.execute("sudo modprobe nvidia")

Verifiy services

In [ ]:
my_server.execute("nvidia-smi")
my_server.execute("""
if [ -L /usr/local/bin/nvcc ]; then sudo rm /usr/local/bin/nvcc; fi && \
sudo ln -s /usr/local/cuda-11.2/bin/nvcc /usr/local/bin/nvcc
""")
my_server.execute("nvcc --version")

## Run Fairify

Now, we can finally run Fairify. First we need to clone the github repo first and then run the reprduce.sh script which contain the instructions from README.md
packaged into a bash file

In [37]:
my_server.execute("rm -rf Fairify && git clone https://github.com/radhofan/Fairify.git")

Cloning into 'Fairify'...


<Result cmd='rm -rf Fairify && git clone https://github.com/radhofan/Fairify.git' exited=0>

### Run dependencies and first experiment

In [38]:
my_server.execute("chmod +x Fairify/reproduce-experiment.sh")
my_server.execute("bash Fairify/reproduce-experiment.sh")

ERROR: File or directory already exists: '/home/cc/miniconda'
If you want to update an existing installation, use the -u option.


Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.

Running `shell init`, which:
 - modifies RC file: "/home/cc/.bashrc"
 - generates config for root prefix: "/home/cc/miniconda"
 - sets mamba executable to: "/home/cc/miniconda/bin/mamba"
The following has been added in your "/home/cc/.bashrc" file

# >>> mamba initialize >>>
# !! Contents within this block are managed by 'mamba shell init' !!
export MAMBA_EXE='/home/cc/miniconda/bin/mamba';
export MAMBA_ROOT_PREFIX='/home/cc/miniconda';
__mamba_setup="$("$MAMBA_EXE" shell hook --shell bash --root-prefix "$MAMBA_ROOT_PREFIX" 2> /dev/null)"
if [ $? -eq 0 ]; then
    eval "$__mamba_setup"
else
    alias mamba="$MAMBA_EXE"  # Fallback on help from mamba activate
fi
unset __mamba_setup
# <<< mamba initialize <<<



warning  libmamba 'repo.anaconda.com', a commercial channel hosted by Anaconda.com, is used.
    
warning  libmamba Please make sure you understand Anaconda Terms of Services.
    
warning  libmamba See: https://legal.anaconda.com/policies/en/




Transaction

  Prefix: /home/cc/miniconda/envs/fairify

  Updating specs:

   - python=3.9


  Package               Version  Build           Channel         Size
───────────────────────────────────────────────────────────────────────
  Install:
───────────────────────────────────────────────────────────────────────

  + _libgcc_mutex           0.1  main            pkgs/main     Cached
  + _openmp_mutex           5.1  1_gnu           pkgs/main     Cached
  + ca-certificates   2025.2.25  h06a4308_0      pkgs/main     Cached
  + ld_impl_linux-64       2.40  h12ee557_0      pkgs/main     Cached
  + libffi                3.4.4  h6a678d5_1      pkgs/main     Cached
  + libgcc-ng            11.2.0  h1234567_1      pkgs/main     Cached
  + libgomp              11.2.0  h1234567_1      pkgs/main     Cached
  + libstdcxx-ng         11.2.0  h1234567_1      pkgs/main     Cached
  + ncurses                 6.4  h6a678d5_0      pkgs/main     Cached
  + openssl              3.0.16  h5eee18b_0      

Reading package lists...
Building dependency tree...
Reading state information...
csvtool is already the newest version (2.4-1build3).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Started running verification for Fairify/src/DF/Verify-DF.py models.


...

       True
2        True
3       False
4        True
        ...  
4495    False
4496     True
4497    False
4498     True
4499    False
Length: 4500, dtype: bool
True: 1648 | False: 2852
prot_attr
0       0.0
1       1.0
2       1.0
3       0.0
4       1.0
       ... 
4495    1.0
4496    0.0
4497    1.0
4498    1.0
4499    0.0
Length: 4500, dtype: float64
INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Expected input size: 30, Actual input size: 30
Expected input size: 30, Actual input size: 30
Verifying ...
unknown
Pruning done!
0.0 % HEURISTIC PRUNING
0.0 % TOTAL PRUNING
Expected input size: 30, Actual input size: 30
Expected input size: 30, Actual input size: 30
Verifying ...
unknown
V time:  100.006
******************
21th column
0       0.0
1       1.0
2       1.0
3       0.0
4       1.0
       ... 
4495    1.0
4496    0.0
4497    1.0
4498    1.0
4499    0.0
Name: 20, Length: 4500, dtype: float64
y_true
0       0
1       0
2       0
3       0
4       1
       ..
4495 

  File "<frozen importlib._bootstrap>", line 984, in _find_and_load_unlocked


9    0.0
Name: 20, Length: 4500, dtype: float64
y_true
0       0
1       0
2       0
3       0
4       1
       ..
4495    0
4496    1
4497    0
4498    1
4499    0
Length: 4500, dtype: int64
True: 1003 | False: 3497
y_pred
0        True
1        True
2        True
3       False
4        True
        ...  
4495    False
4496     True
4497    False
4498     True
4499    False
Length: 4500, dtype: bool
True: 1648 | False: 2852
prot_attr
0       0.0
1       1.0
2       1.0
3       0.0
4       1.0
       ... 
4495    1.0
4496    0.0
4497    1.0
4498    1.0
4499    0.0
Length: 4500, dtype: float64
INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Expected input size: 30, Actual input size: 30
Expected input size: 30, Actual input size: 30
Verifying ...
unknown
Pruning done!
0.0 % HEURISTIC PRUNING
0.0 % TOTAL PRUNING
Expected input size: 30, Actual input size: 30
Expected input size: 30, Actual input size: 30
Verifying ...
unknown
V time:  100.97
******************
21th column
0   

ModuleNotFoundError: No module named 'utils.DF-4-Model-Functions'


<Result cmd='bash Fairify/reproduce-experiment.sh' exited=0>

Results for first experiment

In [39]:
my_server.execute("csvtool readable Fairify/src/DF/DF-1.csv | less -S")

Partition_ID Verification SAT_count UNSAT_count UNK_count h_attempt h_success B_compression S_compression ST_compression H_compression T_compression SV-time S-time HV-Time H-Time Total-Time C-check V-accurate Original-acc Pruned-acc Acc-dec C1 C2
1            unknown      0         0           1         1         0         0.0           0.0           0.0            0.0           0.0           100.014 100.61 100.064 100.34 200.95     0       0          0.6091       1.0        -          
2            unknown      0         0           2         1         0         0.0           0.0           0.0            0.0           0.0           100.295 100.94 100.023 100.29 201.23     0       0          0.6091       1.0        -          
3            unknown      0         0           3         1         0         0.0           0.0           0.0            0.0           0.0           100.774 101.36 100.148 100.42 201.78     0       0          0.6091       1.0        -          
4            unkno

<Result cmd='csvtool readable Fairify/src/DF/DF-1.csv | less -S' exited=0>

In [40]:
my_server.execute("csvtool readable Fairify/src/DF/synthetic-default-predicted-DF-1-metrics.csv | less -S")

Partition ID Original Accuracy  Original F1 Score   Pruned Accuracy Pruned F1 DI                 SPD                  EOD                AOD                  ERD                  CNT          TI
1            0.6091111111111112 0.33647680120709167 1.0             1.0       1.0312860436654625 0.011315487014541603 0.0086764897215717 0.008923965698996894 0.009861184718996219 [0.91062222] 0.18992487920379048
2            0.6091111111111112 0.33647680120709167 1.0             1.0       1.0312860436654625 0.011315487014541603 0.0086764897215717 0.008923965698996894 0.009861184718996219 [0.91062222] 0.18992487920379048
3            0.6091111111111112 0.33647680120709167 1.0             1.0       1.0312860436654625 0.011315487014541603 0.0086764897215717 0.008923965698996894 0.009861184718996219 [0.91062222] 0.18992487920379048
4            0.6091111111111112 0.33647680120709167 1.0             1.0       1.0312860436654625 0.011315487014541603 0.0086764897215717 0.008923965698996894 0.009861184

<Result cmd='csvtool readable Fairify/src/DF/synthetic-default-predicted-DF-1-metrics.csv | less -S' exited=0>